## Normal shading with textures

This example shows how to:
   - modify predefined materials
   - use displacement map
   - apply large textures to material
   
Tkinter GUI window is launched from the notebook in this example. This allows re-running code cells and see results without scrolling back to the figure.

![notebook output image](https://plotoptix.rnd.team/images/normal_shading_with_textures.jpg "This notebook output")

In [1]:
import numpy as np
from plotoptix import TkOptiX
from plotoptix.materials import m_clear_glass, m_plastic # predefined materials
from plotoptix.utils import map_to_colors, make_color_2d, read_image, simplex
from plotoptix.enums import RtFormat # texture buffer formats

Make some data first.

In [2]:
m = 300
r = 0.08 * np.random.rand(m) + 0.01
p = 3 * (np.random.rand(m,3) - 0.5)
p[:,0] *= 0.8
p[:,2] *= 1.5
p[:,1] = r[:]

Setup the raytracer using Tkinter GUI as the output target.

In [3]:
optix = TkOptiX()
optix.set_param(min_accumulation_step=4,    # set more accumulation frames to get rid of the noise
                max_accumulation_frames=512)
optix.set_uint("path_seg_range", 5, 10)     # more path segments to allow multiple reflections

Only *diffuse* material is available by default. Other materials need to be configured before using.

In [4]:
m_beads = m_plastic.copy() # a new material based on predefined properties

optix.setup_material("plastic", m_plastic)
optix.setup_material("glass", m_clear_glass)
optix.setup_material("beads", m_beads)

Add objects to the scene.

**Note 1:** ``geom_attr="ModulatedNormal"`` is used to allow for shading the plane object with displacement data.

**Note 2:** particles with ``geom="ParticleSetTextured"`` geometry can have 3D orientation, provided with ``u`` and ``v`` arguments. 3D oreintation is randomized if these vectors are ommited.

In [5]:
optix.set_data("plane", geom="Parallelograms", geom_attr="ModulatedNormal", mat="plastic",
               pos=[-3, 0, -3], u=[6, 0, 0], v=[0, 0, 6], c=0.8)

optix.set_data("sphere", geom="ParticleSetTextured", geom_attr="ModulatedNormal", mat="glass",
               pos=[0.0, 0.7, 0.0], u=[-1, 0, 0], r=0.4, c=10)

optix.set_data("particles", geom="ParticleSetTextured", mat="beads",
               pos=p, r=r, c=0.95)

Setup a good point of view, set background and lights. Use ligth shading best for caustics.

In [6]:
optix.setup_camera("cam1", cam_type="DoF",
                   eye=[-2.1, 2.4, 0], target=[0, 0, 0], up=[0.28, 0.96, 0.05],
                   aperture_radius=0.01, fov=30, focal_scale=0.91)

optix.set_light_shading("Hard")
optix.setup_light("light1", pos=[4, 5.1, 3], color=[12, 11, 10], radius=1.9)
optix.setup_light("light2", pos=[-1.5, 3, -2], color=[8, 9, 10], radius=0.2)
optix.set_background(0)
optix.set_ambient(0)

exposure = 0.4; gamma = 2.2 
optix.set_float("tonemap_exposure", exposure)
optix.set_float("tonemap_igamma", 1 / gamma)
optix.add_postproc("Gamma")  # apply gamma correction postprocessing stage, or
#optix.setup_denoiser()      # use AI denoiser (exposure and gamma are applied as well)

Open the GUI.

In [7]:
optix.start()

Make a gradient texture, use it with the *glass* material:

In [8]:
y = np.linspace(0, 1, 100)
M = np.stack((y,y)).T

In [9]:
M1 = map_to_colors(-M, "Purples")
M1 = make_color_2d(M1, gamma=gamma, channel_order="RGBA")
m_clear_glass["Textures"]=[
        {
          "Width": M1.shape[1],
          "Height": M1.shape[0],
          "DataArray": (10*M1).flatten().tolist(),  # glass color has the attenuation lenght meaning
          "Format": RtFormat.Float4.value
        }
      ]
m_clear_glass["VarFloat3"]["refraction_index"] = [1.4, 1.4, 1.4]
optix.update_material("glass", m_clear_glass, refresh=True)

Make a gradient with another color map for the plastic particles.

In [10]:
M2 = map_to_colors(M, "RdYlBu")
M2 = make_color_2d(M2, gamma=gamma, channel_order="RGBA")
m_beads["Textures"]=[
        {
          "Width": M2.shape[1],
          "Height": M2.shape[0],
          "DataArray": M2.flatten().tolist(),
          "Format": RtFormat.Float4.value
        }
      ]
optix.update_material("beads", m_beads, refresh=True)

Calculate displacement map over 2D plane using simplex noise:

In [11]:
nn = 2000
x = np.linspace(0, 50, nn)
z = np.linspace(0, 50, nn)

X, Z = np.meshgrid(x, z)
XZ = np.stack((X.flatten(), Z.flatten(), np.full(nn**2, 1.0, dtype=np.float32))).T.reshape(nn, nn, 3)
XZ = np.ascontiguousarray(XZ, dtype=np.float32)
Y = simplex(XZ)
Y = np.sin(10 * Y)

Use the displacement for the shading normal modulation. Displacement is relative to the object size, it should be a small value to look like a wrinkles on the plane.

In [12]:
optix.set_displacement("plane", 0.00008*Y, refresh=True)

OK, to make it more interesting, let's add some lines along wrinkles.

**Note:** large textures would take long time to go through JSON serialization/deserialization; it is much faster to define a small texture at the beginning and update it with actual data later.

In [13]:
m_plastic["Textures"]=[
        {
          "Width": 2,
          "Height": 2,
          "DataArray": np.full(2*2*4, 0.8).tolist(),
          "Format": RtFormat.Float4.value
        }
      ]
optix.update_material("plastic", m_plastic)

In [14]:
Ym = 0.5 * (np.copy(Y) + 1)
m = (Ym > 0.45) & (Ym < 0.55)
Ym[m] = 0.0
Ym[~m] = 0.95

In [15]:
M3 = make_color_2d(Ym, channel_order="RGBA")
optix.update_material_texture("plastic", M3, refresh=True)

Close GUI window, release resources.

In [16]:
optix.close()